In [1]:
from main import *
from sklearn.model_selection import train_test_split

import keras
import tensorflow as tf
from tensorflow.keras import layers, models

SHORT = True

2024-10-28 16:27:18.518145: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-28 16:27:18.531867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730129238.543581    5348 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730129238.547198    5348 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 16:27:18.561303: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
if SHORT == True:
    machines = ["M01", "M02"]
    process_names = ["OP02","OP07"]
else:
    machines = ["M01", "M02","M03"]
    process_names = ["OP00","OP01","OP02","OP03","OP04","OP05","OP06","OP07","OP08","OP09","OP10","OP11","OP12","OP13","OP14"]

labels = ["good","bad"]
path_to_dataset = Path("./data/").absolute()


In [3]:
X_data = []
y_data = []

for process_name, machine, label in itertools.product(process_names, machines, labels):
    data_path = os.path.join(path_to_dataset, machine, process_name, label)
    data_list, data_label = data_loader_utils.load_tool_research_data(data_path, label=label, add_additional_label = True, verbose = False)
    #concatenating
    X_data.extend(data_list)
    y_data.extend(data_label)

In [4]:
X = [x.astype(np.float64) for x in X_data] # Ensure all examples have the same datatype
X = np.array([x[:4096, :3] for x in X]) # Select the first 4096 timesteps of all three dimensions for each row
y = np.array([0 if id.split("_")[-1] == "good" else 1 for id in y_data])

In [6]:
from sklearn.preprocessing import MinMaxScaler
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size = 0.5, stratify = y_trainval)

X_train.shape, X_val.shape, X_test.shape

scaler = MinMaxScaler()

T = [identity, cut_paste, mean_shift, missing_signal]

X_train = augment(T, X_train, 5)
X_val = augment(T, X_val, 5)
X_test = augment(T, X_test, 5)

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Processing <function identity at 0x7f5708a540e0> transformations
Processing <function cut_paste at 0x7f5708a54180> transformations
Processing <function mean_shift at 0x7f5708a54220> transformations
Processing <function missing_signal at 0x7f5708a542c0> transformations


NameError: name 'X_augmented_arr' is not defined

In [ ]:
# Define the model f
def create_f_model():
    model = models.Sequential()
    model.add(layers.InputLayer(shape=(128, 512, 3)))

    model.add(layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(32))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(weight_decay = 10^3),
    metrics=["f1_score"],
    )
    return model

def create_g_loc_model():
    model = models.Sequential()
    
    model.add(layers.Dense(16))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(8))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(6, activation='softmax'))

    return model

def create_g_trans_model():
    model = models.Sequential()

    model.add(layers.Dense(16))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(8))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(4, activation='softmax'))

    return model

# Creating the models
f_model = create_f_model()
g_loc_model = create_g_loc_model()
g_trans_model = create_g_trans_model()

# Print model summaries
f_model.summary()
g_loc_model.summary()
g_trans_model.summary()


In [ ]:
def create_f_model_downstream():
    model = models.Sequential()
    model.add(layers.InputLayer(shape=(128, 512, 3)))

    model.add(layers.Conv2D(8, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(16))
    model.add(layers.LeakyReLU())

    # Hotelling's T^2 output (Dense layer with 1 output neuron)
    model.add(layers.Dense(1, name="hotellings_T2"))
    return model

# Create the model
f_model_downstream = create_f_model_downstream()

# Print the model summary
#f_model_downstream.summary()